In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import pandas as pd

# Initialize Selenium WebDriver (adjust the path to your chromedriver)
service = Service('path_to_chromedriver')
driver = webdriver.Chrome(service=service)

# Function to scrape news using Selenium
def scrape_news_with_selenium():
    driver.get('https://www.bbc.com/news')
    
    # Let the page load and get the page source
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # List to store scraped news data
    news_data = []

    # Scrape the first 5 articles
    articles = soup.find_all('a', class_='gs-c-promo-heading', limit=5)
    
    if not articles:
        print("No articles found. The CSS selectors may have changed.")
        return []

    for article in articles:
        title = article.get_text()
        link = 'https://www.bbc.com' + article['href']
        
        # Visit each article link
        driver.get(link)
        article_soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        # Scrape article text
        paragraphs = article_soup.find_all('p')
        text = ' '.join([p.get_text() for p in paragraphs])
        
        news_data.append({
            'title': title,
            'text': text,
            'label': 1 # Assign a label later or classify
        })

    return news_data

# Scrape the news using Selenium
scraped_news = scrape_news_with_selenium()

# Close the browser after scraping
driver.quit()

# Process scraped data (same as before)
if scraped_news:
    scraped_news_df = pd.DataFrame(scraped_news)
    data = pd.read_csv('WELFake_Dataset.csv')
    data.drop('Unnamed: 0', axis=1, inplace=True)
    data = pd.concat([data, scraped_news_df], ignore_index=True)
    data.to_csv('WELFake_Dataset_updated_with_scraped_news.csv', index=False)
    print(data.tail(10))
else:
    print("No news articles were scraped.")


NoSuchDriverException: Message: Unable to obtain driver for chrome; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location
